# Aufgabe-C-Sturm

## Allgemeines

Eine allgemeine Beschreibung der Laboraufgaben inklusive des Vorgehens, den Bewertungsrichtlinien und der Abgabe finden Sie  <a href="ML-allgemein.ipynb">hier</a>

## Datenquelle


* Laden Sie ihre Daten von http://141.72.190.207/ml_lab/C_sturm herunter
    * Die Daten sind geschützt. 
        * Sie müssen evtl. in einem Netzwerk der DHBW (z.B. WLAN, VPN, ...) angemeldet sein. 
        * Sie können sich auf der Webseite mit dem Benutzernamen dhbw und dem Zugangsnamen "ml_LaB_4$" anmelden. 
* Die Daten sind in einem anwendungsspezifischen Format gespeichert.
    * Sie finden evtl. Informationen über die Daten in einer "README" Datei. 
    * Finden Sie keine solche Datei sind die Daten selbst erklärend. 
    
    



## Aufgabe

Sagen Sie die Windgeschwindigkeit eines Sturms (in Knoten) aufgrund von Satellitenfotos vorher.
Der Datensatz besteht aus Fotos von 494 verschiedenen Stürmen im Atlantik und Pazifik mit ihren zugehörigen Windgeschwindigkeiten.
Jedes Bild hat `366 x 366` Pixel, und es sind 70.257 Trainingsdaten und 45.377 Testdaten vorhanden.
Die Bilder wurden zu mehreren Zeitpunkten während der Lebensdauer eines Sturms aufgenommen.

Für jeden Sturm im Trainings und Testdatensatz erhalten Sie eine Zeitreihe von Bildern mit der jeweiligen assoziierten relativen Zeit seit Beginn des Sturms.
Ihr Modell sollte neben den reinen Bilddaten also auch den zeitlichen Verlauf des Sturms betrachten, um die Vorhersage für künftige Vorhersagezeitpunkte zu erstellen

Die Bilder sind nach folgendem Schema benannt: `{image_id}.jpg`.
Diese IDs bestehen aus einer Sturm_ID und einer Bildnummer entsprechend der zeitlichen Bildreihenfolge.

Ihr Ziel ist es, für die Testdaten die korrekte Windgeschwindigkeiten vorherzusagen.

# Lösung

* Beginnen Sie hier mit Ihrer Dokumentation und Implementierung! 

Geplantes Vorgehen: 

- Daten vorbereitung
- CNN-Modell trainieren (für räumliche Merkmale der Bilder)
- LSTM-Modell trainieren (für zeitlichen Verlauf der Bilder)
- Kombinierung der Modelle (Fully Connected-Layer um CNN in LSTM einzuspeisen, Ausgabe von LSTM ist finale Vorhersage)
- Kombinierte Modell auf Trainingsdaten trainieren 

--------------------------------------------------------------------------------------------------------------------------------------------------------------

Das geplante Vorgehen für die Vorhersage von Windgeschwindigkeiten eines Sturms aufgrund von Satellitenfotos mithilfe von tiefen neuronalen Netzen kann wie folgt aussehen:

1. Datenanalyse: Zunächst sollten die Trainings- und Testdaten untersucht werden, um eine Vorstellung davon zu bekommen, wie die Daten strukturiert sind und wie die Windgeschwindigkeiten mit den Bilddaten und dem zeitlichen Verlauf des Sturms korrelieren.

2. Datenpräparation: Die Bilddaten können normalisiert und in eine geeignete Form gebracht werden, um sie für das Training des neuronalen Netzes zu verwenden. Dabei kann auch die Information über den zeitlichen Verlauf des Sturms in die Datensätze integriert werden.

3. Modellierung: Es kann ein tiefes neuronales Netzwerk entworfen werden, das die Bilddaten und die Information über den zeitlichen Verlauf des Sturms als Eingabe erhält und die Windgeschwindigkeit als Ausgabe generiert. Hierbei können verschiedene Architekturen von neuronalen Netzen ausprobiert werden, um das beste Ergebnis zu erzielen.

4. Training: Das entworfene Modell kann mit den Trainingsdaten trainiert werden. Dabei sollten geeignete Verlustfunktionen und Optimierer gewählt werden, um die Leistung des Modells zu verbessern.

5. Evaluierung: Nach dem Training sollte die Leistung des Modells mit den Testdaten ausgewertet werden, um zu überprüfen, ob es in der Lage ist, die Windgeschwindigkeiten korrekt vorherzusagen.

6. Feintuning: Wenn das Modell nicht zufriedenstellend abschneidet, kann es durch Feintuning verbessert werden, indem z.B. die Hyperparameter optimiert werden.

7. Anwendung: Wenn das Modell gut funktioniert, kann es für Vorhersagen von Windgeschwindigkeiten von Stürmen aufgrund von Satellitenfotos verwendet werden.

Es ist zu beachten, dass es bei jedem Schritt des oben beschriebenen Vorgehens viele Möglichkeiten und Entscheidungen gibt, die je nach den Eigenschaften der Daten und des Modells angepasst werden können.

In [1]:
import sys
from pathlib import Path
module_path = str(Path.cwd() / "src")
if module_path not in sys.path:
    sys.path.append(module_path)

In [23]:
import os
import json
import numpy as np
from PIL import Image
from truncate import truncate

train_data_dir = 'data\\storm_train_data'
train_labels_dir = 'data\\storm_train_labels'
test_data_dir = 'data\\storm_test_data'
test_labels_dir = 'data\\storm_test_labels'

img_size = (366,366)
seq_length = 10

# Funktion um die Daten für ein einzelnes Bild zu laden und zu vorverarbeiten
def load_image(file_path):
    img = Image.open(file_path).convert('L') # oeffnet das Bild und konvertiert in Graustufen
    img = img.resize(img_size) # skaliert das Bild auf die gewuenschte groesse
    img_array = np.array(img) # stellt das Bild als Pixelarray dar
    img_array = img_array / 255.0 # normalisiert das Pixelarray auf das Intervall [0,1]
    return img_array
    
# Funktion um Sequentiell die Daten der Stürme als auch die Labels zu laden
def load_storm_sequences(data_dir, label_dir, limit = -1):
    if limit == -1: 
        limit = len(os.listdir(data_dir))
    img_seq, label_seq, img_sequences, label_sequences = [], [], [], []
    i =0
    for storm_dir in os.listdir(data_dir):
        i += 1
        print(f'{progress(i, limit)} %')
        img = load_image(os.path.join(data_dir,storm_dir,'image.jpg'))
        # img_seq.append(img)
        img_sequences.append(img)
        with open(os.path.join(label_dir,storm_dir,'labels.json')) as f:
            labels = json.load(f)
            label_sequences.append(int(labels["wind_speed"]))
            # for key in labels.keys():
            #     if(key != "wind_speed"): continue
        #         label_seq.append(labels[key])
        # if(i%10==0):
        #     img_sequences.append(img_seq)
        #     label_sequences.append(label_seq)
        #     img_seq, label_seq =[],[]
        if(i > limit-1): break
    return np.array(img_sequences), np.array(label_sequences)

def progress(index, len):
    return truncate((index/len)*100,3)

def load_data():
    train_img_sequences, train_label_sequences = load_storm_sequences(train_data_dir, train_labels_dir, 15000)
    test_img_sequences, test_label_sequences = load_storm_sequences(test_data_dir, test_labels_dir,5000)
    # train_img_sequences = np.concatenate(train_img_sequences, axis =0)
    # train_label_sequences = np.concatenate(train_label_sequences, axis =0)
    # test_img_sequences = np.concatenate(test_img_sequences, axis=0)
    # test_label_sequences = np.concatenate(test_label_sequences, axis=0)
    return train_img_sequences, train_label_sequences, test_img_sequences, test_label_sequences

# lädt die Daten aufgeteilt in Test und Trainingsdaten in Arrays
X_train, Y_train, X_test, Y_test = load_data()

0.006 %
0.013 %
0.02 %
0.026 %
0.033 %
0.04 %
0.046 %
0.053 %
0.06 %
0.066 %
0.073 %
0.08 %
0.086 %
0.093 %
0.1 %
0.106 %
0.113 %
0.12 %
0.126 %
0.133 %
0.139 %
0.146 %
0.153 %
0.16 %
0.166 %
0.173 %
0.18 %
0.186 %
0.193 %
0.2 %
0.206 %
0.213 %
0.22 %
0.226 %
0.233 %
0.24 %
0.246 %
0.253 %
0.26 %
0.266 %
0.273 %
0.279 %
0.286 %
0.293 %
0.3 %
0.306 %
0.313 %
0.32 %
0.326 %
0.333 %
0.339 %
0.346 %
0.353 %
0.36 %
0.366 %
0.373 %
0.38 %
0.386 %
0.393 %
0.4 %
0.406 %
0.413 %
0.42 %
0.426 %
0.433 %
0.44 %
0.446 %
0.453 %
0.459 %
0.466 %
0.473 %
0.48 %
0.486 %
0.493 %
0.5 %
0.506 %
0.513 %
0.52 %
0.526 %
0.533 %
0.54 %
0.546 %
0.553 %
0.559 %
0.566 %
0.573 %
0.58 %
0.586 %
0.593 %
0.6 %
0.606 %
0.613 %
0.62 %
0.626 %
0.633 %
0.64 %
0.646 %
0.653 %
0.66 %
0.666 %
0.673 %
0.679 %
0.686 %
0.693 %
0.7 %
0.706 %
0.713 %
0.72 %
0.726 %
0.733 %
0.74 %
0.746 %
0.753 %
0.76 %
0.766 %
0.773 %
0.779 %
0.786 %
0.793 %
0.8 %
0.806 %
0.813 %
0.82 %
0.826 %
0.833 %
0.84 %
0.846 %
0.853 %
0.86 %
0.866 %
0.87

--------------------------------------------------------------------------------
Die Daten sind nun in die Arrays geladen. Aufgeteilt in trainings-daten und trainings-labels, sowie test-daten und test-labels.
Die Bilddateien werden auf eine einheitliche Größe skaliert und anschliessend werden die Pixel in einem Array normalisiert.

In [24]:
print(f'Menge der Trainingsdaten (Daten zu Labels): {len(X_train)} zu {len(Y_train)}')
print(f'Menge der Testdaten (Daten zu Labels): {len(X_test)} zu {len(Y_test)}')

Menge der Trainingsdaten (Daten zu Labels): 15000 zu 15000
Menge der Testdaten (Daten zu Labels): 5000 zu 5000


In [25]:
np.shape(X_train)

(15000, 366, 366)

In [11]:
# X_train = X_train.reshape(X_train.shape[0],366,366,1)
# X_test = X_test.reshape(X_test.shape[0],366,366,1)

In [26]:
np.shape(Y_train)

(15000,)

---------------------------------------------------------------------------------------------------------------
Die Datenvorverarbeitung ist beinahe abgeschlossen.
Die Trainings- und Testdaten liegen jetzt sequentiell in 10er Sequenzen in Numpy Arrays vor.
Da es sich bei der Aufgabe um ein Regressionsproblem handelt und die Labels bereits als reelle Zahlen (Windgeschwindigkeiten) vorliegen, müssen die Labeldaten nicht weiter vorbereitet werden.
Die Aufgabenstellung sieht vor Tensorflow zu nutzen, somit muss das Paket erst einmal in unserem Kernel installiert werden.
Der Code ist auskommentiert um eine unnötige mehrmalige Ausführung zu vermeiden

In [14]:
# !pip install tensorflow

     ---------------------------------------- 0.0/266.3 MB ? eta -:--:--
     -------------------------------------- 0.0/266.3 MB 653.6 kB/s eta 0:06:48
     ---------------------------------------- 0.1/266.3 MB 1.1 MB/s eta 0:04:14
     ---------------------------------------- 0.1/266.3 MB 1.2 MB/s eta 0:03:40
     -------------------------------------- 0.2/266.3 MB 952.6 kB/s eta 0:04:40
     -------------------------------------- 0.2/266.3 MB 888.4 kB/s eta 0:05:00
     -------------------------------------- 0.2/266.3 MB 888.4 kB/s eta 0:05:00
     -------------------------------------- 0.3/266.3 MB 785.2 kB/s eta 0:05:39
     -------------------------------------- 0.3/266.3 MB 873.8 kB/s eta 0:05:05
     -------------------------------------- 0.4/266.3 MB 794.9 kB/s eta 0:05:35
     -------------------------------------- 0.4/266.3 MB 819.2 kB/s eta 0:05:25
     -------------------------------------- 0.5/266.3 MB 861.1 kB/s eta 0:05:09
     -------------------------------------- 0.6

Nun muss ein CNN Modell definiert werden, dass in der Lage ist, aus den Bilddaten räumliche Merkmale zu extrahieren.

In [27]:
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential()
model.add(layers.Conv2D(32, (3,3), activation ='relu', input_shape =(366,366,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(64,activation ='relu'))
model.add(layers.Dense(10, activation = 'softmax'))

model.compile(optimizer ='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Um die Trainingsdaten in Trainings- und Validierungsdaten zu trennen wählen wir die Methode train_test_split der Bibliothek scikit-learn.
Diese muss ebenfalls erst installiert werden

In [30]:
# !pip install -U scikit-learn

     ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/8.4 MB 660.6 kB/s eta 0:00:13
     ---------------------------------------- 0.1/8.4 MB 1.1 MB/s eta 0:00:08
      --------------------------------------- 0.1/8.4 MB 1.1 MB/s eta 0:00:08
      --------------------------------------- 0.2/8.4 MB 1.0 MB/s eta 0:00:08
     - -------------------------------------- 0.3/8.4 MB 1.1 MB/s eta 0:00:08
     - -------------------------------------- 0.3/8.4 MB 1.2 MB/s eta 0:00:07
     - -------------------------------------- 0.4/8.4 MB 1.2 MB/s eta 0:00:07
     -- ------------------------------------- 0.5/8.4 MB 1.2 MB/s eta 0:00:07
     -- ------------------------------------- 0.5/8.4 MB 1.2 MB/s eta 0:00:07
     -- ------------------------------------- 0.6/8.4 MB 1.2 MB/s eta 0:00:07
     -- ------------------------------------- 0.6/8.4 MB 1.2 MB/s eta 0:00:07
     --- ------------------------------------ 0.7/8.4 MB 1.2 MB/s eta

In [31]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model

# Definieren der Eingabeformate für die Modelle
input_shape_cnn = (366, 366, 1)  # (Bildhöhe, Bildbreite, Anzahl der Farbkanäle)
input_shape_lstm = (None, 64)   # (Zeitschritte, Anzahl der Features)

# CNN Modell definieren
def create_cnn_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='linear')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# LSTM Modell definieren
def create_lstm_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = LSTM(64)(inputs)
    x = Dense(32, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Datenvorbereitung
# Trainingsdaten und Labels einlesen und vorverarbeiten
# ...

# Datenaufteilung in Trainings- und Validierungsdaten
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# CNN Modell instanziieren und kompilieren
cnn_model = create_cnn_model(input_shape_cnn)
cnn_model.compile(optimizer='adam', loss='mse')

# LSTM Modell instanziieren und kompilieren
lstm_model = create_lstm_model(input_shape_lstm, num_classes=10)
lstm_model.compile(optimizer='adam', loss='categorical_crossentropy')

# Training des CNN Modells
history_cnn = cnn_model.fit(X_train, Y_train, batch_size=32, epochs=10, validation_data=(X_val, Y_val))

# Training des LSTM Modells
# history_lstm = lstm_model.fit(x_train_sequence, y_train_wind_speed, batch_size=32, epochs=10, validation_data=(x_val_sequence, y_val_wind_speed))

Epoch 1/10
375/375 [==============================] - 1686s 4s/step - loss: 517.1224 - val_loss: 333.7847
Epoch 2/10
375/375 [==============================] - 1716s 5s/step - loss: 285.5677 - val_loss: 160.6040
Epoch 3/10
375/375 [==============================] - 1716s 5s/step - loss: 176.5472 - val_loss: 123.5254
Epoch 4/10
375/375 [==============================] - 1596s 4s/step - loss: 121.5325 - val_loss: 87.3577
Epoch 5/10
375/375 [==============================] - 1595s 4s/step - loss: 96.0593 - val_loss: 73.5003
Epoch 6/10
375/375 [==============================] - 1583s 4s/step - loss: 81.3259 - val_loss: 66.6385
Epoch 7/10
375/375 [==============================] - 1570s 4s/step - loss: 74.4189 - val_loss: 64.6349
Epoch 8/10
375/375 [==============================] - 1546s 4s/step - loss: 68.3971 - val_loss: 96.2909
Epoch 9/10
375/375 [==============================] - 1511s 4s/step - loss: 60.5135 - val_loss: 60.1306
Epoch 10/10
375/375 [==============================] - 16

In [33]:
model_path = 'models\\cnn_model_1'
cnn_model.save(model_path)

INFO:tensorflow:Assets written to: models\cnn_model_1\assets


INFO:tensorflow:Assets written to: models\cnn_model_1\assets


Das CNN-Modell wurde mit 10 Epochen trainiert und abgespeichert. 
Als nächstes muss das Modell evaluiert werden.